# Finnhub API
Recuperando US stock data de API financiera.

## Readme
Para correr este notebook es necesario un API Key de Finnhub. Puede ser configurado en un archivo
`.env` bajo la carpeta `api-etl`, con nombre de variable `FINNHUB_API_TOKEN`.
También es posible cargarla como variable de entorno del sistema opertivo.


## Imports

In [59]:
import requests
import os
import pandas as pd
import psycopg2
import datetime

## Load ENVs

In [60]:
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## API Class

In [61]:
# La API tiene una libreria de Python, pero vamos a armar nuestra propia versión (más acorde a la consigna).

class Finnhub:
  BASE_URL="https://finnhub.io/api/v1"

  def __init__(self, token):
    self.token = token
  
  def quote(self, symbol):
    params = { "symbol": symbol, "token": self.token }
    res = requests.get(f'{self.BASE_URL}/quote', params=params)
    res.raise_for_status()
    return res.json()

Probando la API con el simbolo de Apple

In [62]:
api = Finnhub(token=os.environ['FINNHUB_API_TOKEN'])
api.quote("AAPL")

{'c': 165.84,
 'd': 0.84,
 'dp': 0.5091,
 'h': 167.26,
 'l': 164.77,
 'o': 165.75,
 'pc': 165,
 't': 1713816001}

## Extraer y Guardar

Definimos nuestras acciones deseadas

In [63]:
stocks = [
  {'name': 'Apple', 'symbol': 'AAPL'},
  {'name': 'Google', 'symbol': 'GOOGL'},
  {'name': 'Coca Cola', 'symbol': 'KO'},
  {'name': 'Microsoft', 'symbol': 'MSFT'},
  {'name': 'Amazon', 'symbol': 'AMZN'},
  {'name': 'JP Morgan', 'symbol': 'JPM'},
  {'name': 'Spotify', 'symbol': 'SPOT'},
  {'name': 'Disney', 'symbol': 'DIS'},
  {'name': 'Tesla', 'symbol': 'TSLA'},
  {'name': 'Intel', 'symbol': 'INTC'}
]

Conseguimos el precio de cada una, y lo guardamos en un array

In [64]:
results = []

In [65]:
for stock in stocks:
  data = api.quote(stock['symbol'])
  result = {k: data[k] for k in ('c', 'h', 'l', 'o', 'pc', 't')}
  result = {**stock, **result}
  results.append(result)
  print(f"{stock['name']}: ${data['c']:,.2f}")

Apple: $165.84
Google: $156.28
Coca Cola: $60.55
Microsoft: $400.96
Amazon: $177.23
JP Morgan: $189.41
Spotify: $272.24
Disney: $111.99
Tesla: $142.05
Intel: $34.41


Convertir datos a DF

In [66]:
df = pd.DataFrame(results)
df.rename(columns={'name': 'company', 'c': 'current_price', 'h': 'high_price', 'l': 'low_price', 'o': 'open_price', 'pc': 'last_close_price', 't': 'price_timestamp_unix'}, inplace=True)

Categorizar columnas

In [67]:
df.company = df.company.astype(str)
df.symbol = df.symbol.astype(str)
df.dtypes

company                  object
symbol                   object
current_price           float64
high_price              float64
low_price               float64
open_price              float64
last_close_price        float64
price_timestamp_unix      int64
dtype: object

Agregamos un formato fecha legible

In [68]:
df['price_timestamp'] = pd.to_datetime(df.price_timestamp_unix, unit='s')
df.dtypes

company                         object
symbol                          object
current_price                  float64
high_price                     float64
low_price                      float64
open_price                     float64
last_close_price               float64
price_timestamp_unix             int64
price_timestamp         datetime64[ns]
dtype: object

## Resultado Final

In [69]:

df.drop_duplicates(subset=['symbol', 'price_timestamp'], keep='first', inplace=True)
df[['company', 'symbol', 'current_price', 'high_price', 'low_price', 'open_price', 'last_close_price', 'price_timestamp']]



,company,symbol,current_price,high_price,low_price,open_price,last_close_price,price_timestamp
0,Apple,AAPL,165.84,167.260,164.7700,165.7500,165.00,2024-04-22 20:00:01
1,Google,GOOGL,156.28,157.640,154.0600,154.1100,154.09,2024-04-22 20:00:01
2,Coca Cola,KO,60.55,60.630,59.7150,60.1400,60.17,2024-04-22 20:00:02
3,Microsoft,MSFT,400.96,402.840,395.7500,400.1880,399.12,2024-04-22 20:00:00
4,Amazon,AMZN,177.23,178.860,174.5600,176.9299,174.63,2024-04-22 20:00:02
5,JP Morgan,JPM,189.41,190.125,185.9800,185.9900,185.80,2024-04-22 20:00:02
6,Spotify,SPOT,272.24,280.970,267.7600,280.9700,275.83,2024-04-22 20:00:02
7,Disney,DIS,111.99,113.640,111.7500,113.1000,112.61,2024-04-22 20:01:55
8,Tesla,TSLA,142.05,144.440,138.8025,140.6020,147.05,2024-04-22 20:00:01
9,Intel,INTC,34.41,34.565,34.0750,34.4000,34.20,2024-04-22 20:00:01


Conexión a Redshift

In [70]:
try:
    conn = psycopg2.connect(
        host=os.environ['REDSHIFT_HOST'],
        dbname='data-engineer-database',
        user=os.environ['REDSHIFT_USER'],
        password=os.environ['REDSHIFT_PASSWORD'],
        port='5439'
    )
    print('Conectado a Redshift con éxito!')
    
except Exception as e:
    print('No es posible conectar a Redshift')
    print(e)

Conectado a Redshift con éxito!


Crear tabla si no existe

In [76]:
with conn.cursor() as cur:
    cur.execute("""
        create table if not exists stock_history(
            symbol varchar(100) not null, 
            company varchar(100),
            current_price	float,
            high_price float,
            low_price	float,
            open_price float,
            last_close_price float,
            price_timestamp timestamp not null ,
            created_at timestamp,
            primary key (symbol, price_timestamp)
            );
    """)
    conn.commit()

Insertar registros si no existen

In [75]:
with conn.cursor() as cur:

    for index, row in df.iterrows():
        # Verificar si ya existe el registro
        cur.execute(
            "SELECT 1 FROM stock_history WHERE symbol = %s AND price_timestamp = %s",
            (row["symbol"], row["price_timestamp"]),
        )
        already_exists = cur.fetchone()
        
        if already_exists is None:
            # Ejecutar insert
            cur.execute(
                """INSERT INTO stock_history(
                                symbol,
                                company,
                                current_price,
                                high_price,
                                low_price,
                                open_price,
                                last_close_price,
                                price_timestamp,
                                created_at) VALUES %s
    """,
                [
                    (
                        row["symbol"],
                        row["company"],
                        row["current_price"],
                        row["high_price"],
                        row["low_price"],
                        row["open_price"],
                        row["last_close_price"],
                        row["price_timestamp"].isoformat(timespec="seconds"),
                        datetime.datetime.now(datetime.UTC).isoformat(timespec="seconds"),
                    )
                ],
            )
    conn.commit()

In [73]:
# Cerrar conexion
conn.close()